<a href="https://colab.research.google.com/github/hannape/Gruba-kreska/blob/main/2_Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Będzie to funkcja która pozwoli wczytać dane z pliku wav, podzieli je na M-ms klipy (500 ms domyślne) z N-sek overlapem (150 ms), wczyta etykiety z pliku txt (jeśli więcej niż 4ms głosu w próbce - to pozytywna), stworzy wektory 

* X_train i Y_train
* X_valid i Y_valid

W sumie mam 94 nagrania. 

* treningowy - 88 nagrań, balansujemy, jeśli są dostępne inne etykiety niż interesujących nas głosów (np gh, t, g), to dodajemy je jako próbki negatywne. Jeśłi nie ma głosów w nagraniu to bierzemy P (50) próbek losowych z tego nagrania. 
* walidacyjny to 8 nagrań, które balansujemy tylko trochę - np z poziomu 1:100, do poziomu 1:10.



**Ale najpierw, trzeba jakoś wybrać te nagrania do walida. trzeba sprawdzić stosunki s : k głównie, może jakoś uwzględnić d i r, choć ich dużo mniej, więc z inną wagą**





In [1]:
# Wczytywanie google drive'a
from google.colab import drive
drive.mount('/content/drive')
from __future__ import print_function

Mounted at /content/drive


In [2]:
import os
import contextlib
import numpy as np
import wave
import math
import pandas as pd

In [247]:
# Parametry które będą wchodzić do funkcji

path_test1618_txt = 'drive/My Drive/poprawione etykiety 24112020/testowe/'
path_train161718_txt =  'drive/My Drive/poprawione etykiety 24112020/treningowe/'
path_test1618_wav = 'drive/My Drive/testowe_1618/'
path_train161718_wav =  'drive/My Drive/treningowe94_161718/'
  
balance_types = ['full_rec','balanced', 'valid' ]  # na potrzeby treningu, valida, testu
balance_ratios = [None, [50, 25], [400, 100]]
chunk_length_ms = 500 # 500 ms
chunk_overlap = 150 # 150 ms
calls_0 = ['t', 'g', 'czapla', 'gh', 'puszczyk']                                    # etykiety które są negatywne
calls_1 = ['d', 'd?', 'k', 'k?', 'kwiczol', 'r','r?', 's', 's?', 'skowronek', 'ni'] # calls of interest
calls_unknown = ['???','??? mysz', '??? high freq']                                 # unknown, nie wiem czy to głos czy nie, więc będę wyrzucać te chunksy to zawierające
tolerance = 0.004                                                                   # jaka tolerancja, w sek (jeśli mniej niż ta wartość znajdzie się w chunksie, to uznajemy że głosu tam nie ma)
valid_set = ['9niski_szum_BUK4_20161025_000604', 'BUK5_20180930_000704', 'BUK4_20161024_223604', 'BUK4_20171001_020404a', 'BUK1_20160914_011604', 'BUK5_20170910_025605', 'BUK4_20161013_200104', 'BUK5_20181003_235705']
test_rec_to_cut = ['BUK5_20161101_002104a', 'BUK5_20161101_002104b']  # nagrania do wyrzucenia ze starego zbioru testowego

In [66]:
def my_read_labels(label_name,path_txt):  # ta sama funkcja co w 1. poprawianie danych
    '''Wczytuje etykiety czasowe z pliku labels.txt w folderze train.
    
    Returns:
        ndarray: Tablica z etykietami czasowymi zawierająca kolumny:  sekunda początku dźwięku, sekunda końca dźwięku, gatunek.
    '''
    labels = []
    with open(os.path.join(path_txt, label_name + '.txt'), 'r') as file:
        text = file.read()
        for line in text.split('\n')[0:]:
            if len(line) > 1:
                start, stop, spec = line.split('\t')
                #print(start)
                start, stop, spec = float(start), float(stop), str(spec),
                labels.append([start, stop, spec])
    return np.array(labels)


def my_check_labels(second, chunk_length_s, labels, tol=0.):
    '''Sprawdza czy w ramce czasowej [second, second + chunk_length_s] znajduje się coś wg etykiet `labels`.
    
    Args:
        second (float): Sekunda nagrania.
        labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a 2ga - koniec nagrania.
        tol (float): Tolerancja na brzegach fragmentu. Dźwięk, żeby był uznany, musi się kończyć po czasie `second+tol`
            lub zaczynać przed czasem `second+chunk_length-tol`.
    Returns:
        bool: Czy w ramce czasowej jest etykieta - co najmniej 4 ms labelu.
 ''' 

  # jeśli początek etykiety zawiera się w chunku, najpóźniej 4ms przed końcem chunka
  # jeśli koniec etykiety zawiera się w chunku, i najwcześniej 4 ms po rozpoczęciu chunka
  # jeśli początek etykiety zaczyna się rpzed, a koniec po (w przypadku głosów dłuższych niż chunk length, b. rzadkie przypadki dla 0.5 sek)
    return (float(labels[0]) >= second and float(labels[0]) < second + chunk_length_s - tol) or \
           (float(labels[1]) < second + chunk_length_s and float(labels[1]) > second + tol) or \
           (float(labels[0]) < second and float(labels[1]) > second + chunk_length_s)              
           #and (labels[2] in calls_to_cut)    # nie sprawdzajmy tu jaka to etykieta jest

def my_map_seconds_to_y(labels, recording_duration, calls_of_interest, calls_to_cut, calls_unknown):  ## Chcemy 500 ms z 150 ms overlapem
    '''Tworzy etykiety dla każdego kolejnego chunksa. 1 gdy co najmniej 4 ms etykiety się znajdują w chunksie (gdy urwane dźwięki na brzegach, <4ms - to 0).
    
    Args:
        labels (ndarray): Tablica z etykietami, której 1 kolumna oznacza początek, a druga - koniec nagrania.
    Returns:
        ndarray: Tablica z binarnymi etykietami.
    '''
    #STARE: calls_to_cut = ['t', 't?', 't ','t  ', 'g', 'czapla', 'gh', 'g cz', 'puszczyk','gaski','g?','mewa?','zwierzak?','high freq','g niskie','??? mysz']
    
    
    duration_in_ms = recording_duration*1000
    nr_of_chunks =  1 + (duration_in_ms - chunk_length_ms) / (chunk_length_ms - chunk_overlap)
    print("ilość chunksów w tym nagraniu: ", math.ceil(nr_of_chunks) )
    
    
    y = [0] * math.ceil(nr_of_chunks)             # recording_duration 
    y_restrictive = [0] * math.ceil(nr_of_chunks) # recording_duration
    chunks_start, chunks_end = [0] * math.ceil(nr_of_chunks), [0] * math.ceil(nr_of_chunks)      
    has_unknown, has_noise = [0] * math.ceil(nr_of_chunks), [0] * math.ceil(nr_of_chunks)
    call_id = [[] for _ in range(math.ceil(nr_of_chunks))]
    chunks_species = [[] for _ in range(math.ceil(nr_of_chunks))]
    
    print(np.shape(labels))
    for s in range(math.ceil(nr_of_chunks)):
        chunks_start[s] = s * ((chunk_length_ms-chunk_overlap) / 1000)  # czyli s * 0.35
        chunks_end[s] = chunks_start[s] + chunk_length_ms/1000 
        for ind,l in enumerate(labels):
            #print('label_index: ',ind, ' a label to: ', l[2] )
            if my_check_labels(chunks_start[s], chunk_length_ms/1000, l):
                #print(s*0.35)
                if l[2] in calls_to_cut:
                  has_noise[s] = 1
                if l[2] in calls_unknown:
                  has_unknown[s] = 1
                if l[2] in calls_of_interest:  
                  y[s] = 1 
 
            if my_check_labels(chunks_start[s], chunk_length_ms/1000, l, 0.004): #z tolerancją 
                
                if l[2] in calls_to_cut:
                  chunks_species[s].append(l[2])
                if l[2] in calls_unknown:
                  has_unknown[s] = 1
                  chunks_species[s].append(l[2])
                if l[2] in calls_of_interest:  
                  y_restrictive[s] = 1  
                  chunks_species[s].append(l[2])  
                  call_id[s].append(ind)
              
                      
        if y[s] != y_restrictive[s] and l[2] in calls_of_interest:
            print('próbka ', s, 'zaczynajaca sie ', chunks_start[s],', jest unknown, bo za mały fragment głosu nas interesującego')
            #print('y[s]:', y[s],'yres:', y_restrictive[s] )
            y[s] = 0    # jeśli mniej niż 4 ms fragment, to zakładamy że nie ma głosu, i dajemy unknown także, by nie karać za ewentualne znalezienie tego głosu
            has_unknown[s] = 1
    print('Rozmiar ',np.shape([s, chunks_start, chunks_end, y, chunks_species, call_id, has_unknown]))
    return s+1, chunks_start, chunks_end, y, chunks_species, call_id, has_unknown, has_noise    


def my_load_wav(path_wav, path_txt, recordings_incl = None, recordings_excl = None):   # dłuższa funkcja niż w 1.

  recording_labels_all = []
  in_which_recording = []
  file_names = []

  # wszystkie nagrania
  if recordings_incl == None and recordings_excl == None:                                        
    rec_files = [file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')]

  # nagrania tylko te w podanym zbiorze (walid)
  if recordings_incl != None:                                                                     
    rec_files_all = [file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')]
    recordings_incl = [s + '.wav' for s in recordings_incl]
    rec_files = list( set.intersection(set(rec_files_all), set(recordings_incl)))

  # nagrania oprócz nagrań w podanym zbiorze (train)  
  if recordings_excl != None:                                                                     
    rec_files_all = [file_name for file_name in os.listdir(path_wav) if file_name.endswith('.wav')]
    recordings_excl = [s + '.wav' for s in recordings_excl]
    rec_files = list(set(rec_files_all) - set(recordings_excl))  
 
  print(np.shape(rec_files))
  print((rec_files))
  #print((rec_files))
  #print(np.size(rec_files))

  X_matrix = [[] for _ in range(np.size(rec_files))]
  file_names = [[] for _ in range(np.size(rec_files))]

  for ind, file_name in enumerate(rec_files):
      
      print("------------Analiza nagrania: " + file_name + "-----------")
      recording_id = str(file_name.split('.')[0])
      
      fname = path_wav + file_name      #print(fname)
      with contextlib.closing(wave.open(fname,'r')) as f:
          frames = f.getnframes()
          rate = f.getframerate()
          duration = frames / float(rate)
          recording_duration = math.ceil(duration)
          print("Czas trwania nagrania - w sekundach: " + str(recording_duration))
          #print("rate:", rate)
    
      recording_id = (file_name.split('.')[0])       #print(recording_id)
      recording_labels = my_read_labels(recording_id, path_txt)    

      print("Duration:", duration)
      #y_binary = my_map_seconds_to_y(recording_labels, duration)
      chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown, has_noise  = my_map_seconds_to_y(recording_labels, duration, calls_1, calls_0, calls_unknown)
      chunk_ids = range(chunk_id)
      chunk_start = rate*np.array(chunk_start)
      chunk_start = [round(num) for num in chunk_start]
      chunk_end = rate*np.array(chunk_end)
      chunk_end = [round(num) for num in chunk_end]
      X_matrix[ind] = [chunk_ids, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown, has_noise]  # !!!! chunk start i end przerzucone na sample !!!!!
      file_names[ind] = file_name

  return X_matrix , file_names #chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown

def find_ratio(recordings_names, path_txt):
  count_s,count_k, count_d, count_r, count_positives = 0,0,0,0,0
  for rec_name in recordings_names:
    recording_labels = my_read_labels(rec_name, path_txt)  
    #print(recording_labels)
    counted_labels = Counter(recording_labels[:,2])

    #print(counted_labels)
    for calls in labels_s:
      count_s += counted_labels[calls]

    for calls in labels_k:
      count_k += counted_labels[calls]

    for calls in labels_d:
      count_d += counted_labels[calls]   

    for calls in labels_r:
      count_r += counted_labels[calls] 

    for calls in calls_1:
      count_positives += counted_labels[calls] 

  return [count_s, count_k, count_d, count_r, count_positives]  

In [ ]:
#chunk_id, chunk_start, chunk_end, has_bird, chunks_species, call_id, has_unknown = my_load_wav(path_train161718_wav, path_train161718_txt) 
X_train, file_names_train = my_load_wav(path_train161718_wav, path_train161718_txt, None, valid_set) 
X_valid, file_names_valid = my_load_wav(path_train161718_wav, path_train161718_txt, valid_set, None) 

In [69]:
#print(np.shape(X_train))
#czas_trwania_spr= np.array(X_train[0][1])- np.array(X_train[0][2])
#print(np.unique(czas_trwania_spr)) # upewnienie się że wszystkie chunki jednakowej długości są

columns_dataframe = ['chunk_ids', 'chunk_start', 'chunk_end', 'has_bird', 'chunks_species', 'call_id', 'has_unknown', 'has_noise']

result_dataframe_train = pd.DataFrame(data = X_train, index=file_names_train, columns = columns_dataframe)
result_dataframe_valid = pd.DataFrame(data = X_valid, index=file_names_valid, columns = columns_dataframe)

#print(result_dataframe_train['call_id'][0])
#print(result_dataframe_train.loc[ ['BUK5_20181007_001906.wav'] , ['has_unknown', 'has_noise'] ])
#file_names_train

**Stworzenie zbalansowanego odpowiednio traina i walida**

Więc w trainie, ida wszystkie próbki które mają has_bird = 1 - jako pozytywne

Oprócz tego idą te które mają has_noise, has_bird !=1 i has_unknown - obojetenie - jako negatywne

Następnie pozbywamy się tych z etykietami unknown - by nie poszły przypadkiem do zbioru, i z pozostałych losujemy randomowo próbki bez głosu. 

W trainie - tak jak wcześniej może, tyle głosów negatywnych co pozytywnych, ale minimum 50. Jeśli jest wystarczająco tych z labelem noise, to co najmniej 30 (?) próbek wybranych losowo też




In [261]:
import random

balance_types = ['full_rec','balanced', 'valid' ]
balance_ratios = [None, [50, 25], [400, 100]]  # co najmniej ile próbek negatywnych, w tym ile minimalnie randomów

def create_set_ind(result_dataframe, balance, ratios = None):
  
  print(balance)
  file_names = list(result_dataframe.index)
  print(len(file_names))
  ind_chosen = [[] for _ in range(len(file_names))]
  neg_random_numb = 0;
  all_pos = 0
  all_chunks = 0


  for file_i, file_name in enumerate(file_names):
    print(file_i, ' nagranie ', file_name)
    
    indices_hasbird = [i for i, j in enumerate(result_dataframe['has_bird'][file_i]) if j == 1]
    indices_hasnoise = [i for i, j in enumerate(result_dataframe['has_noise'][file_i]) if j == 1]
    indices_unknown = [i for i, j in enumerate(result_dataframe['has_unknown'][file_i]) if j == 1]
    indices_all = result_dataframe['chunk_ids'][file_i]
    pos = indices_hasbird


    if balance == 'full_rec':
      
      ind_chosen[file_i] = list(set(indices_all) - (set(indices_unknown) - set(indices_hasbird)) )  
      # te które mają ptaka i unknown, to zostają. W każdym innym przypadku, unknown wylatuje z setu
      #print(np.shape(ind_chosen))
      all_pos += len(pos)
      all_chunks += len(ind_chosen[file_i])
      print('pos: ', len(indices_hasbird))  # pozytywne
      neg_labels = list(set(indices_hasnoise) - set(indices_hasbird +indices_unknown))
      print('neg hałas:' , len(neg_labels))
      print('ind_chosen:' , len(ind_chosen[file_i]))

    if balance == 'balanced' or balance == 'valid':
      
      print('pos: ', len(indices_hasbird))  # pozytywne
      
      neg_labels = list(set(indices_hasnoise) - set(indices_hasbird +indices_unknown))
      print('neg hałas:' , len(neg_labels))
      #print(len(list(set(indices_hasnoise) - set(indices_hasbird +indices_unknown)))) # z samym hałasem, bez unknown czy ptaków. Czyli 0
      #temp =  list(set(indices_hasbird + indices_hasnoise))

      
      
      if (len(pos) - len(neg_labels)>=0):   
        neg_random_numb = len(pos) - len(neg_labels)
        if (neg_random_numb < ratios[1]):
          neg_random_numb = ratios[1]
        if neg_random_numb + len(neg_labels) < ratios[0]:
          neg_random_numb = ratios[0] - len(neg_labels)
      else:
        neg_random_numb = ratios[1]

      print('neg random: ', neg_random_numb)

      neg_random_set = list(set(indices_all) - (set(indices_unknown + indices_hasbird + indices_hasnoise)))


      random.seed(file_i)
      neg_random = random.sample(neg_random_set, neg_random_numb)
      print(neg_random[0:5])
      ind_chosen[file_i] = sorted(list(set(indices_hasbird + neg_labels + neg_random)))

      all_pos += len(pos)
      all_chunks += len(ind_chosen[file_i])  

  print('wszystkich chunków: ', all_chunks,', w tym ', all_pos, ' pozytywnych. Czyli ', 100* round(all_pos/all_chunks,4),' % pozytywnych w zbiorze', balance )
  return file_names, ind_chosen


In [246]:
file_names_train_set, ind_for_train_set = create_set_ind(result_dataframe_train, balance_types[1], balance_ratios[1])
file_names_valid_set, ind_for_valid_set = create_set_ind(result_dataframe_valid, balance_types[2], balance_ratios[2])

balanced
86
0  nagranie  BUK5_20181007_001906.wav
pos:  196
neg hałas: 14
neg random:  182
[3285, 3578, 344, 2183, 4375]
1  nagranie  BUK4_20161002_235805.wav
pos:  64
neg hałas: 28
neg random:  36
[1125, 4746, 524, 2125, 988]
2  nagranie  BUK4_20161104_041504.wav
pos:  12
neg hałas: 27
neg random:  25
[468, 755, 700, 2975, 1395]
3  nagranie  BUK4_20171022_004304a.wav
pos:  119
neg hałas: 17
neg random:  102
[1996, 4986, 4567, 1093, 3101]
4  nagranie  BUK4_20160926_004604.wav
pos:  115
neg hałas: 9
neg random:  106
[1989, 2550, 860, 3327, 4018]
5  nagranie  BUK5_20170920_013105b.wav
pos:  36
neg hałas: 35
neg random:  25
[2115, 2978, 4413, 239, 3872]
6  nagranie  BUK4_20161104_034504.wav
pos:  2
neg hałas: 17
neg random:  25
[4719, 663, 3987, 2150, 303]
7  nagranie  BUK5_20171007_031905.wav
pos:  119
neg hałas: 34
neg random:  85
[2702, 1251, 3316, 397, 596]
8  nagranie  BUK4_20160925_191604.wav
pos:  33
neg hałas: 35
neg random:  25
[1865, 3064, 3105, 1038, 1590]
9  nagranie  BUK4_201

In [217]:
# sprawdzenie tego w pełnych dataFrame'ach train i walid, czy zsumują się tak samo wartości z kolumny has_bird
check_sum_positives = 0

for i,file_name_train_set in enumerate(file_names_train_set):
  rowData = result_dataframe_train.loc[file_name_train_set]
  check_sum_positives += np.array(rowData["has_bird"])[ind_for_train_set[i]].sum()
print(check_sum_positives)

check_sum_positives = 0
for i,file_name_valid_set in enumerate(file_names_valid_set):
  rowData = result_dataframe_valid.loc[file_name_valid_set]
  check_sum_positives += np.array(rowData["has_bird"])[ind_for_valid_set[i]].sum()

print(check_sum_positives)

7198
214


In [250]:
# No i sprawdzenie testu, jaki tam stosunek będzie jeśli wyrzucimy próbki unknown ( i sprawdzic ile ich jest)
X_test_new, file_names_test_new = my_load_wav(path_test1618_wav, path_test1618_txt, None, test_rec_to_cut) 
result_dataframe_test_new = pd.DataFrame(data = X_test_new, index=file_names_test_new, columns = columns_dataframe)

X_test_old, file_names_test_old = my_load_wav(path_test1618_wav, path_test1618_txt) 
result_dataframe_test_old = pd.DataFrame(data = X_test_old, index=file_names_test_old, columns = columns_dataframe)

tuuu!
(18,)
['BUK4_20160922_005604.wav', 'BUK4_20180913_201703.wav', 'BUK5_20181021_004605deszczyk.wav', 'BUK4_20161008_004605.wav', 'BUK5_20181010_011205.wav', 'BUK5_20181028_043105.wav', 'BUK1_20181013$023504.wav', 'BUK5_20180921_015906a.wav', 'BUK1_20181011$001004.wav', 'BUK5_20181103_011804.wav', 'BUK4_20161016_035704.wav', 'BUK5_20180909_010005.wav', 'BUK5_20181014_233005.wav', 'BUK5_20180921_015906b.wav', 'BUK5_20181020_014805a.wav', 'BUK1_20180918$040704.wav', 'BUK4_20180925_224703.wav', 'BUK4_20161016_012704.wav']
------------Analiza nagrania: BUK4_20160922_005604.wav-----------
Czas trwania nagrania - w sekundach: 1799
Duration: 1798.002358276644
ilość chunksów w tym nagraniu:  5137
(40, 3)
próbka  1461 zaczynajaca sie  511.34999999999997 , jest unknown, bo za mały fragment głosu nas interesującego
Rozmiar  (7,)
------------Analiza nagrania: BUK4_20180913_201703.wav-----------
Czas trwania nagrania - w sekundach: 1800
Duration: 1799.0008163265306
ilość chunksów w tym nagraniu:

In [262]:
file_names_test_set_old, ind_for_test_set_old = create_set_ind(result_dataframe_test_old, balance_types[0], balance_ratios[0])
file_names_test_set_new, ind_for_test_set_new = create_set_ind(result_dataframe_test_new, balance_types[0], balance_ratios[0])

full_rec
20
0  nagranie  BUK4_20160922_005604.wav
pos:  58
neg hałas: 10
ind_chosen: 5135
1  nagranie  BUK4_20161016_035704.wav
pos:  15
neg hałas: 19
ind_chosen: 5131
2  nagranie  BUK5_20161101_002104b.wav
pos:  401
neg hałas: 28
ind_chosen: 5123
3  nagranie  BUK4_20161016_012704.wav
pos:  0
neg hałas: 10
ind_chosen: 5137
4  nagranie  BUK4_20161008_004605.wav
pos:  13
neg hałas: 37
ind_chosen: 5119
5  nagranie  BUK5_20161101_002104a.wav
pos:  160
neg hałas: 46
ind_chosen: 5113
6  nagranie  BUK5_20181020_014805a.wav
pos:  59
neg hałas: 54
ind_chosen: 5125
7  nagranie  BUK1_20181013$023504.wav
pos:  54
neg hałas: 71
ind_chosen: 5129
8  nagranie  BUK5_20180921_015906b.wav
pos:  73
neg hałas: 18
ind_chosen: 5128
9  nagranie  BUK5_20181103_011804.wav
pos:  8
neg hałas: 72
ind_chosen: 5133
10  nagranie  BUK5_20181028_043105.wav
pos:  30
neg hałas: 57
ind_chosen: 5123
11  nagranie  BUK1_20181011$001004.wav
pos:  201
neg hałas: 61
ind_chosen: 5125
12  nagranie  BUK4_20180925_224703.wav
pos:  

In [267]:
print(len(result_dataframe_test_new['chunk_ids'][0]))
all_chunks_chosen =0
all_chuks_ever = 0

for i in range(18):
  all_chuks_ever += len(result_dataframe_test_new['chunk_ids'][i])
  all_chunks_chosen +=  len(ind_for_test_set_new[i])
#.sum()
print(all_chunks_chosen)
print(all_chuks_ever)
print('Chunksy unknown, różnica: ', all_chuks_ever - all_chunks_chosen)

5137
92326
92441
Chunksy unknown, różnica:  115


In [ ]:
#@title
# do sprawdzenia czy te zagnieżdżone listy w call_id i chunks_species działają

def isListEmpty(inList):
  for elem in inList:
    if elem: # Is a list
        print(elem)
        #return all( map(isListEmpty, inList) )
    #return False # Not a list

isListEmpty(result_dataframe['call_id'][2])
isListEmpty(result_dataframe['chunks_species'][2])


In [ ]:
#@title
# do sprawdzenia jakie ID mają głosy w naszym nagraniu (każdy z labeli ma ID, nie tylko głosy, dlatego nie wszystkie liczby tu będą)

test_list = result_dataframe['call_id'][2]

#print("The original list : " + str(test_list)) 
  
res = [] 
temp = set() 
for inner in test_list: 
        for ele in inner: 
            if not ele in temp: 
                temp.add(ele) 
                res.append(ele) 
  
print("Unique elements in nested tuples are : " + str(res)) 

--------------------------------------------------------------------------------

**Wybór nagrań do walida**

In [ ]:
rec_for_train = [ 181006, 181007, 181001, 181002, 161103, 161104, 160925, 160926, 161031, 161005, 160918 ] 
labels_s = ['s', 's?']
labels_k = ['k', 'k?']
labels_d = ['d', 'd?']
labels_r = ['r', 'r?']
calls_1 = ['d', 'd?', 'k', 'k?', 'kwiczol', 'r','r?', 's', 's?', 'skowronek', 'ni']

In [ ]:
# Podstawowy wybór nagrań treningowych i walidacyjnych praz zbiór testowy: stare 20 nagrań, nowe 18 nagrań

all_rec_dates_train, all_rec_dates_test = [],[]
potential_valid_rec = []
basic_train_rec = []
test_new_rec,test_old_rec = [], []

for file_name in file_names_train:

  recording_date = str(file_name.split('_20')[1][0:6])
  all_rec_dates_train.append(int(recording_date))
  if int(recording_date) not in rec_for_train:
    potential_valid_rec.append(file_name.split('.')[0])
  else:
    basic_train_rec.append(file_name.split('.')[0]) 
    
print(all_rec_dates_train)
print(potential_valid_rec)
print(basic_train_rec)

for file_name in file_names_test:

  recording_date = str(file_name.split('_20')[1][0:6])
  all_rec_dates_test.append(int(recording_date))
  test_old_rec.append(file_name.split('.')[0])
  if int(recording_date) != 161101:
    test_new_rec.append(file_name.split('.')[0])

print(test_new_rec)
print(test_old_rec)

In [ ]:
count_ratio_valid = find_ratio(potential_valid_rec, path_train161718_txt)  
count_ratio_train = find_ratio(basic_train_rec, path_train161718_txt)
count_ratio_test_old= find_ratio(test_old_rec, path_test1618_txt)  # wszystkie 20 nagrań
count_ratio_test_new= find_ratio(test_new_rec, path_test1618_txt)  # bez tych 2 zawierających dużo k, czyli 18 nagrań

print('Ratio s : k : d : r')
print('Valid (31): ', count_ratio_valid[0],' : ',count_ratio_valid[1],' : ',count_ratio_valid[2],' : ',count_ratio_valid[3],'. Ratio s:k - ', count_ratio_valid[0]/count_ratio_valid[1])  
print('Train (63): ', count_ratio_train[0],' : ',count_ratio_train[1],' : ',count_ratio_train[2],' : ',count_ratio_train[3], '. Ratio s:k - ', count_ratio_train[0]/count_ratio_train[1])  
print('Test old (20): ', count_ratio_test_old[0],' : ',count_ratio_test_old[1],' : ',count_ratio_test_old[2],' : ',count_ratio_test_old[3], '. Ratio s:k - ', count_ratio_test_old[0]/count_ratio_test_old[1])  
print('Test new (18): ',count_ratio_test_new[0],' : ',count_ratio_test_new[1],' : ',count_ratio_test_new[2],' : ',count_ratio_test_new[3], '. Ratio s:k - ', count_ratio_test_new[0]/count_ratio_test_new[1])  

#print(np.shape(potential_valid_dates))
#print(np.shape(basic_train_rec))

In [ ]:
import random

## funkcja losująca 8 nagrań do walida (z 31), resztę wrzucająca do traina, przeliczenie score
# scorer = stosunek s:k najważniejszy, a tak z 1/5 wagi na d
scorer_best, scorer = 10, 10

for i in range(1000):

  random_valid_rec = random.sample(potential_valid_rec, 8)
  remaining_train_rec = []


  #print(random_valid_rec)

  for file_name in file_names_train:
    #print(file_name)
    if (file_name.split('.')[0]) not in random_valid_rec:
    #print(file_name)
      remaining_train_rec.append(file_name.split('.')[0])

    
  count_ratio_valid_8 = find_ratio(random_valid_rec, path_train161718_txt)
  count_ratio_train63_23 = find_ratio(remaining_train_rec, path_train161718_txt)

  #print(np.shape(remaining_train_rec))
  #print(np.shape(random_valid_rec))
  #print((remaining_train_rec))
  #print((random_valid_rec))
  if (count_ratio_valid_8[2]!=0):
    scorer_kd = 0.2*abs((count_ratio_train63_23[1]/count_ratio_train63_23[2]) - ( count_ratio_valid_8[1]/count_ratio_valid_8[2] ))
  else:
    scorer_kd = 666

  if (count_ratio_valid_8[1]!=0):
    scorer_sk = abs((count_ratio_train63_23[0]/count_ratio_train63_23[1]) - ( count_ratio_valid_8[0]/count_ratio_valid_8[1] ) )
  else:
    scorer_sk = 1000000  

  scorer = scorer_sk + scorer_kd

  print(scorer)
  if scorer_best > scorer:
    print(i)
    print('scorer: ', scorer)
    print('Valid ratio s:k - ', count_ratio_valid_8[0]/count_ratio_valid_8[1])
    print('Train ratio s:k - ', count_ratio_train63_23[0]/count_ratio_train63_23[1])
    print('Valid ratio k:d - ', count_ratio_valid_8[1]/count_ratio_valid_8[2])
    print('Train ratio k:d - ', count_ratio_train63_23[1]/count_ratio_train63_23[2])
    scorer_best =  scorer
    validate_rec_best = random_valid_rec
    count_ratio_valid_best = count_ratio_valid_8
    count_ratio_train_best = count_ratio_train63_23




In [ ]:
print(validate_rec_best)
print('Valid (8): ', count_ratio_valid_best[0],' : ',count_ratio_valid_best[1],' : ',count_ratio_valid_best[2],' : ',count_ratio_valid_best[3],'. Ratio s:k - ', count_ratio_valid_best[0]/count_ratio_valid_best[1], 'Ratio k:d - ', count_ratio_valid_best[1]/count_ratio_valid_best[2], 'Ratio d:r - ', count_ratio_valid_best[2]/count_ratio_valid_best[3])  
print('Train (86): ', count_ratio_train_best[0],' : ',count_ratio_train_best[1],' : ',count_ratio_train_best[2],' : ',count_ratio_train_best[3], '. Ratio s:k - ', count_ratio_train_best[0]/count_ratio_train_best[1], 'Ratio k:d - ', count_ratio_train_best[1]/count_ratio_train_best[2], 'Ratio d:r - ', count_ratio_train_best[2]/count_ratio_train_best[3])  

## Upewnić się że się daty nie pokrywają
validate_rec_best_dates = []
rec_dates_train, rec_dates_valid = [],[]

for file_name in validate_rec_best:
  recording_date = str(file_name.split('_20')[1][0:6])
  validate_rec_best_dates.append(recording_date)

print(validate_rec_best_dates)

for file_name in file_names_train:
  #print(file_name)
  recording_date = str(file_name.split('_20')[1][0:6])
  #print(recording_date)
  if (recording_date) in validate_rec_best_dates:
    rec_dates_valid.append(recording_date)
  else:  # final trening 88
    rec_dates_train.append(recording_date) 
# Nie pokrywają się

#Counter(rec_dates_valid)
#Counter(rec_dates_train)

In [ ]:
### TO JEST OD DZIŚ MÓJ WALID

valid_set = ['9niski_szum_BUK4_20161025_000604', 'BUK5_20180930_000704', 'BUK4_20161024_223604', 'BUK4_20171001_020404a', 'BUK1_20160914_011604', 'BUK5_20170910_025605', 'BUK4_20161013_200104', 'BUK5_20181003_235705']
